# Исследование надёжности заёмщиков

### Цель: узнать влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

### Ответить на поставленные вопросы:
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?

_____

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выгрузка представляет из себя таблицу в 12 колонок и 21525 записями в ней. Имеются разные типы данных. У столбцов days_employed и total_income имеются пропущенные значения. 

### Заполним пропуски

##### Заполним пропуски в доходах. 

In [4]:
ave_income = data['total_income'].mean()
print('Средняя по доходам: {:.0f}'.format(ave_income))

Средняя по доходам: 167422


Так как люди работают на разных работах и на разных должностях, правильнее будет посмотреть среднюю в зависимости от занимаемой должности.

In [5]:
data.groupby('income_type')['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

In [6]:
avr_fam_id_0 = 131339.751676
avr_fam_id_1 = 53829.130729
avr_fam_id_2 = 170898.309923
avr_fam_id_3 = 202417.461462
avr_fam_id_4 = 137127.465690
avr_fam_id_5 = 499163.144947
avr_fam_id_6 = 161380.260488
avr_fam_id_7 = 98201.625314

Заменим пропущенные значения на 0, нам это понадобится для заполнения их нашими средними.

In [7]:
data['total_income'] = data['total_income'].fillna(0)

In [8]:
'''Функция возвращает среднюю в зависимости от должности, 
а так же используется фильтр по 0, что бы заполнить именно там, 
где встречается 0, нашей средней'''

def return_mean_from_i_t_and_t_i (row):
    f_i = row['income_type']
    t_i = row['total_income']
    if f_i == 'безработный':
        if t_i == 0:
            return avr_fam_id_0
    if f_i == 'в декрете':
        if t_i == 0:
            return avr_fam_id_1
    if f_i == 'госслужащий':
        if t_i == 0:
            return avr_fam_id_2
    if f_i == 'компаньон':
        if t_i == 0:
            return avr_fam_id_3
    if f_i == 'пенсионер':
        if t_i == 0:
            return avr_fam_id_4
    if f_i == 'предприниматель':
        if t_i == 0:
            return avr_fam_id_5
    if f_i == 'сотрудник':
        if t_i == 0:
            return avr_fam_id_6
    if f_i == 'студент':
        if t_i == 0:
            return avr_fam_id_7
    return 0

In [9]:
data['avg'] = data.apply(return_mean_from_i_t_and_t_i, axis=1)

На данном этапе мы получили столбец, где пропущенные значения из столбца total_income заменены на средний доход в зависимости от должности, остальные значения равны 0.

In [10]:
data['total_income'] = data['total_income'] + data['avg'].replace()

In [11]:
data = data.drop(columns = 'avg')

Просто сложив два столбца я заполнил уже 0 из total_income. Далее удаляем  avg столбец, он более не нужен.

_______

##### Заполним пропуски в трудовом стаже. 

In [12]:
#data.head()

Изучив несколько строк можно сразу заметить какие-то отрицательные значения и весьма большие числа. Посмотрим на них с точки зрения средней, медианы, максимальных и минимальных значений.

In [13]:
ave_data_mean = data['days_employed'].mean()
ave_data_median = data['days_employed'].median()
ave_data_max = data['days_employed'].max()
ave_data_min = data['days_employed'].min()
print('Средняя в годах',ave_data_mean/365)
print("Медиана в годах",ave_data_median/365)
print("Максимум в годах ",ave_data_max/365)
print("Минимум в годах ",ave_data_min/365)

Средняя в годах 172.73013057937914
Медиана в годах -3.296902818549285
Максимум в годах  1100.6997273296713
Минимум в годах  -50.38068465909146


Да, безумно странные числа, к тому же переведенные в года. Максимум 1100 лет, быть может это Дункан Маклауд, но он нам тут ни к чему. Ни средняя ни медиана не под силу обычному человеку. Для начала уберем все отрицательные числа.

In [14]:
data['days_employed'] = data['days_employed'].abs()

In [15]:
ave_data_mean = data['days_employed'].mean()
ave_data_median = data['days_employed'].median()
ave_data_max = data['days_employed'].max()
ave_data_min = data['days_employed'].min()
print('Средняя в годах',ave_data_mean/365)
print("Медиана в годах",ave_data_median/365)
print("Максимум в годах ",ave_data_max/365)
print("Минимум в годах ",ave_data_min/365)

Средняя в годах 183.32802440225305
Медиана в годах 6.0115631969279315
Максимум в годах  1100.6997273296713
Минимум в годах  0.06614146093282515


Цифры стали приятнее, да, средняя все так же высока, но вот медиана довольно приличная. 

In [16]:
#data['days_employed'] = data['days_employed'].fillna(0)

In [17]:
#data[data['days_employed']==0]['dob_years'].median()

In [18]:
#a= data[data['days_employed']==0]['dob_years']

In [19]:
#a.value_counts()

In [20]:
#data[data['dob_years']==0]['dob_years'].count()

In [21]:
#data[(data['days_employed']==0) & (data['dob_years'] != 0)]['dob_years'].median()

In [22]:
#print(data[data['dob_years']==43]['days_employed'].median() /365)

Выше я проделал некоторые манипуляции, что бы узнать возраст по медиане относительно незаполненных строк в столбце days_employed. Я узнал, что 101 человек не указал свой возраст, а точнее указал как 0 и 10 из них в нашей выборке. Исключив эти десять нулей, я получил средний возраст равный 43, но медиана по рабочим дням этого возраста всего 4 года. Так как возраст довольно разный, принято решение заменить пропущенные значения общей медианой ( 6 лет)

In [23]:
data['days_employed'] = data['days_employed'].fillna(ave_data_median)

In [24]:
#data.head()

In [25]:
#data.info()

____

## Вывод
Пропущенные значения имеются в столбцах days_employed и total_income, они имеют количественные переменные. Пропусков более 10% от данных, я не могу их просто удалить. Логически, если человек не имеет трудового стажа, то он не может подтвердить свой официальный доход. Но, для ответа на поставленный вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" я выбираю установить средний доход, отсортировав его в зависимости от типа занятости. Для пропущенных значений в столбце days_employed было принято решение установить медиану, так как имеются сильно разнящиеся значения, но для начала я привел все отрицательные значения в положительные. 

____

##### Замена вещественного типа данных на целочисленный

In [26]:
data['days_employed'] = data['days_employed'].astype('int')

In [27]:
data['total_income'] = data['total_income'].astype('int')

In [28]:
#data.head()

In [29]:
#data.info()

____

## Вывод
На целочислинный тип данных была переведены колонки days_employed и total_income. При переводе методом to_numeric() мы превратим строки в числовой тип, но он у нас и так уже числовой и вещественный. Необходимо использовать astype('int'), что бы сделать наши колонки целочисленными.

____

##### Поиск и удаление дубликатов

Посмотрим сколько дубликатов на данный момент

In [30]:
data.duplicated().sum()

54

Удалим дубликаты

In [31]:
data = data.drop_duplicates()

В таблице был замечен разный регист в столбце education, убедимся в этом, а так-же поищем есть ли ещё

In [32]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [33]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [34]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [35]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [36]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Заметим, что в столбце education имеется разный регистр, приведем все к единому и проверим на дубликаты.

In [37]:
data['education'] = data['education'].str.lower()

In [38]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [39]:
data.duplicated().sum()

17

In [40]:
data = data.drop_duplicates()

___

# Вывод
Использовав метод duplicated() мы обнаружили 54 полных дубликата. Проанализировав таблицу, можно заметить, что в столбце education имеется разный регистр, исправив это, мы обнаружили еще 17 дубликатов. Все дубликаты удаляем с помощью метода drop_duplicates(). Я выделяю несколько причин появления дубликатов. Во-первых - это техническая. Во-вторых - это, возможно, точно такой же человек.  

_____

##### Выделяем леммы в значениях столбца с целями получения кредита

In [41]:
from pymystem3 import Mystem
m = Mystem()

In [42]:
my_list = data.loc[:,'purpose'].tolist()

In [43]:
text = ','.join(my_list)

In [44]:
lemmas =m.lemmatize(text)

In [45]:
#print(lemmas)

In [46]:
from collections import Counter

In [47]:
print(Counter(lemmas))

Counter({' ': 33570, ',': 21453, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


___

## Вывод
Первым делом был получен список всех целей кредита, далее я его перевел в строку. Выделив леммы я их, так же, подсчитал.

____

##### Категоризация данных и создание словарей

In [48]:
#data.head()

In [49]:
print(data['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Дети либо есть, либо их нет, отрицательных не бывает(по количеству)! А вот 20 бывает, но это скорее редкость. Приведем данные в порядок.

In [50]:
data['children'] = data['children'].abs()

In [51]:
data['children'] = data['children'].replace(20,2)

In [52]:
''' Функция категоризирует семьи по количеству детей'''

def family_category (data):
    children = data['children']
    if children == 0:
        return 'бездетная'
    if children == 1:
        return 'однодетная'
    if children == 2:
        return 'малодетная'
    return 'многодетная'

In [53]:
data['family_category'] = data.apply(family_category, axis =1)

In [54]:
print(data['family_category'].value_counts())

бездетная      14091
однодетная      4855
малодетная      2128
многодетная      380
Name: family_category, dtype: int64


In [55]:
def income_category (data):
    income = data['total_income']
    if income < 12130:
        return 'ниже МРОТ'
    if income < 25000:
        return 'низкий'
    if income < 120000:
        return 'средний'
    if income < 500000:
        return 'выше среднего'
    if income < 1000000:
        return 'высокий'
    return 'более_миллиона'

In [56]:
#data['income_category'] = data.apply(income_category, axis=1)

In [57]:
#print(data['income_category'].value_counts())

In [58]:
data['total_income'].describe()

count    2.145400e+04
mean     1.674316e+05
std      9.806060e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.518870e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [59]:
low_99 =0.99
low_50 = 0.5
low_1 = 0.01

data['total_income'].quantile(q=[low_99,low_50,low_1])

0.99    505511.63
0.50    151887.00
0.01     44967.08
Name: total_income, dtype: float64

In [60]:
def income_category_quantile (data):
    income = data['total_income']
    if income < 45000:
        return 'низкий'
    if income < 152000:
        return 'ниже среднего'
    if income < 505000:
        return 'выше среднего'
    return 'высокий'

In [61]:
data['income_category'] = data.apply(income_category_quantile, axis=1)

In [62]:
print(data['income_category'].value_counts())

ниже среднего    10527
выше среднего    10494
низкий             217
высокий            216
Name: income_category, dtype: int64


Доход разбил на 4 группы квантилями. Проанализировав данные, немного оперевшись на реальность, я отделил 1 % группы людей с высокими и низкими доходами и установил явную середину. Так получились эти 4 группы.

___

##### Категоризируем данные по типу сделки с использованием лемм

In [64]:
data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)

In [65]:
def loan_purpose (data):
    purpose = data['purpose_lemma']
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'cделка с недвижимостью'
    if 'автомобиль'in purpose:
        return 'cделка с автомобилем'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'строительство' in purpose:
        return 'строительство'
    if 'ремонт' in purpose:
        return 'ремонт'
    return 'другое'

In [66]:
data['loan_purpose'] = data.apply(loan_purpose, axis=1)

In [67]:
print(data['loan_purpose'].value_counts())

cделка с недвижимостью    10811
cделка с автомобилем       4306
образование                4013
свадьба                    2324
Name: loan_purpose, dtype: int64


###### Создаем словари

Словарь: "Образование"

In [68]:
data_education_dict = data[['education', 'education_id']]

In [69]:
data_education_dict = data_education_dict.drop_duplicates().reset_index(drop = True)

In [70]:
data_education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Словарь: "Семейное положение"

In [71]:
data_family_status_dict = data[['family_status', 'family_status_id']]

In [72]:
data_family_status_dict = data_family_status_dict.drop_duplicates().reset_index(drop=True)

In [73]:
data_family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [74]:
data = data.drop(columns=['education', 'family_status'])

In [75]:
#data.head()

___

## Вывод
В таблице уже многое категоризированно, к примеру, образование или статус семьи, но я решил выделить еще несколько категорий. Во-первых - это категоризация по количеству детей в семье (взята с какого то сайта :)). В этой категории были приведены данные в порядок. Использовалась логика, что не может быть -1 ребенок, скорее это ошибка записи. Далее хоть и бывают семьи с 20 детьми, но это, скорее, редкость, я это тоже учел как за ошибку и приравнял к 2. Следующая категория, это категории по уровню достатка, квантилями были выделенны 4 группы. В третей категории учитывалась цель кредита. По леммам четко можно проследить несколько повторяющихся целей.
Дополнительно были выделенны 2 словаря "Семейное положение" и словарь "Образование", это позволяет удалить их текстовое наименование из таблицы, а оставть их уникальный id.

____

In [76]:
#data.info()

In [77]:
data_children = data.groupby(['family_category', 'debt']).agg({'debt': 'count'})

In [78]:
print(data_children)

                       debt
family_category debt       
бездетная       0     13028
                1      1063
малодетная      0      1926
                1       202
многодетная     0       349
                1        31
однодетная      0      4410
                1       445


In [79]:
all_no_children_debt = data[data['family_category'] == 'бездетная']['debt'].count()
all_one_children_debt = data[data['family_category'] == 'однодетная']['debt'].count()
all_two_children_debt = data[data['family_category'] == 'малодетная']['debt'].count()
all_lot_children_debt = data[data['family_category'] == 'многодетная']['debt'].count()
print('Общее количесво взятых кредитов у бездетных {}'.format(all_no_children_debt))
print('Общее количесво взятых кредитов у однодетных семей {}'.format(all_one_children_debt))
print('Общее количесво взятых кредитов у малодетных семей {}'.format(all_two_children_debt))
print('Общее количесво взятых кредитов у многодетных семей {}'.format(all_lot_children_debt))

Общее количесво взятых кредитов у бездетных 14091
Общее количесво взятых кредитов у однодетных семей 4855
Общее количесво взятых кредитов у малодетных семей 2128
Общее количесво взятых кредитов у многодетных семей 380


In [80]:
print('Процент возврата кредита у бездетных {:.2%}'.format(13086 / all_no_children_debt))
print('Процент возврата кредита у однодетных семей {:.2%}'.format(4420 / all_one_children_debt))
print('Процент возврата кредита у малодетных семей {:.2%}'.format(1929 / all_two_children_debt))
print('Процент возврата кредита у многодетных семей {:.2%}'.format(349 / all_lot_children_debt))

Процент возврата кредита у бездетных 92.87%
Процент возврата кредита у однодетных семей 91.04%
Процент возврата кредита у малодетных семей 90.65%
Процент возврата кредита у многодетных семей 91.84%


## Вывод
Можно сказать, что чем меньше детей, тем чаще люди берут кредиты, и заметить, что процент возврата у людей не имеющих детей самый высокий. Интересно, что процент возврата в срок у многодетных семей выше, чем у семей с одним или двумя детьми. 

In [81]:
def status_debt (data):
    debt = data['debt']
    if debt == 1:
        return 'невозврат в срок'
    return 'возврат в срок'

In [82]:
data['status_debt'] = data.apply(status_debt, axis=1)

In [89]:
#data.head()

In [84]:
data_children_family_status = data.groupby(['family_status_id','debt']).agg({'debt': 'count'})

In [85]:
data_family_status = data.pivot_table(index=['family_status_id'], columns='status_debt', values='debt', aggfunc='count', fill_value=0, margins=True)

In [86]:
#data_family_status

In [87]:
print(data_family_status['возврат в срок']/(data_family_status['невозврат в срок'] + data_family_status['возврат в срок'])*100)

family_status_id
0      92.454818
1      90.652855
2      93.430657
3      92.887029
4      90.249110
All    91.884963
dtype: float64


In [90]:
data_family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


## Вывод
Можем заметить, что чаще возвращают в срок кредиты люди, которые состоят в статусе вдовец / вдова, далее находящиеся в разводе и тройку замыкают клиенты состоящие в браке. Сохраниться ли этот процент, если разгруппировать по детям. 

In [91]:
data_children_family_status_pivot = data.pivot_table(index=['family_category', 'family_status_id'], columns='status_debt', values='debt', aggfunc='count', fill_value=0, margins=True)

In [ ]:
#data_children_family_status_pivot.reset_index()

In [92]:
print(data_children_family_status_pivot['возврат в срок']/(data_children_family_status_pivot['невозврат в срок'] + data_children_family_status_pivot['возврат в срок'])*100)

family_category  family_status_id
бездетная        0                    93.090520
                 1                    91.611722
                 2                    93.742621
                 3                    92.984694
                 4                    90.716180
малодетная       0                    90.644753
                 1                    90.704225
                 2                    87.500000
                 3                    90.361446
                 4                    88.095238
многодетная      0                    92.982456
                 1                    87.878788
                 2                   100.000000
                 3                    91.666667
                 4                    80.000000
однодетная       0                    91.777630
                 1                    88.200000
                 2                    91.358025
                 3                    93.354430
                 4                    88.546256
All   

## Вывод
В общем, тройка лидеров сохраняется. Но здесь мы можем заметить, как уменьшается процент возврата в срок кредитов у людей не состоящих в браке по отношению к количеству детей. Далее посмотрим, как влияет доход на возврат кредитов в срок.

In [93]:
data_income_pivot = data.pivot_table(index=['income_category'], columns='status_debt', values='debt', aggfunc='count', fill_value=0, margins=True)

In [94]:
#data_income_pivot

In [95]:
print(data_income_pivot['возврат в срок']/(data_income_pivot['невозврат в срок'] + data_income_pivot['возврат в срок'])*100)

income_category
высокий          93.518519
выше среднего    92.138365
ниже среднего    91.564548
низкий           93.548387
All              91.884963
dtype: float64


## Вывод

Интересно, что сейчас хуже всех возвращают в срок люди, которые зарабатывают от 45 до 152 тысяч и далее по возрастающей. Было бы больше данных, к примеру суммы кредита, тогда можно было бы делать более точные выводы, а так можно только догадываться из-за чего такая просадка. И все же можно наблюдать рост % возврата в срок с повышением дохода.

In [96]:
data_purpose_pivot = data.pivot_table(index=[ 'loan_purpose'], columns='status_debt', values='debt', aggfunc='count', fill_value=0, margins=True)

In [97]:
print(data_purpose_pivot['возврат в срок']/(data_purpose_pivot['невозврат в срок'] + data_purpose_pivot['возврат в срок'])*100)

loan_purpose
cделка с автомобилем      90.640966
cделка с недвижимостью    92.766627
образование               90.779965
свадьба                   91.996558
All                       91.884963
dtype: float64


## Вывод
Лучше всего кредиты платятся в срок по недвижимости. Я думаю, это из-за того, что это долгосрочные ипотечные кредиты имеют хорошо рассчитанную по доходу клиента сумму, а так же, не каждый клиент подойдет. Далее идет свадьба, скорее это небольшие кредиты, и их уже возвращают пара молодоженов. А вот кредиты на образование и автомобили возвращаются хуже.  

In [100]:
loan_purpose_pivot = data.pivot_table(index=['family_status_id','loan_purpose'], columns='status_debt', values='debt', aggfunc='count', fill_value=0, margins=True)

In [101]:
#loan_purpose_pivot

In [102]:
print(loan_purpose_pivot['возврат в срок']/(loan_purpose_pivot['невозврат в срок'] + loan_purpose_pivot['возврат в срок'])*100)

family_status_id  loan_purpose          
0                 cделка с автомобилем      91.630117
                  cделка с недвижимостью    93.065068
                  образование               91.676301
1                 cделка с автомобилем      88.248848
                  cделка с недвижимостью    90.798787
                  образование               85.148515
                  свадьба                   91.996558
2                 cделка с автомобилем      90.825688
                  cделка с недвижимостью    94.833948
                  образование               92.462312
3                 cделка с автомобилем      92.526690
                  cделка с недвижимостью    93.047337
                  образование               92.857143
4                 cделка с автомобилем      87.127159
                  cделка с недвижимостью    91.854637
                  образование               89.254766
All                                         91.884963
dtype: float64


В разрезе статуса семьи первенство сохраняется за недвижимостью. Кроме клиентов состоящих в гражданском браке, ее вытесняет желание скорее рассчитаться за недавнюю свадьбу. 

## Вывод

Были проанализированы данные, эти данные прошли предобработку, а именно, были удалены дубликаты и заполнены пропуски, данные были категоризированны и подготовлены к ответам на вопросы исследования. 

Исследование показало:

Если клиент не имеет детей, то в 92.87% кредит возвращается в срок.
Клиент, который находится в браке и платит по кредиту в срок в 92.45%, если в разводе то 92.89% и, если находится в статусе вдовец\вдова, то 93.43% соответственно.
Просматаривается прямая зависимость, чем больше уровень дохода, тем выше процент платежа в срок по кредиту. Но, есть и уникальный всплеск, для людей зарабатывающих менее 45 тысяч рублей в месяц, процент платежа в срок самый высокий, это может быть связанно с суммой кредита. 
А самой платежеспособной целью является недвижимость.

Был выявлен самый благоприятный клиент для банка. 
##### Это человек, который когда-либо состоял или состоит в браке, не имеет детей, доход его свыше 152 тысяч рублей, а целью его кредита является недвижимость.   